# NASA Battery Dataset Analysis

### Introduction:
In this notebook, we analyze the NASA Battery Dataset to observe how key battery parameters evolve over repeated charge and discharge cycles.  
We focus on three key parameters:
1. **Battery Impedance**: Combined internal resistance of the battery.
2. **Re**: Electrolyte Resistance.
3. **Rct**: Charge Transfer Resistance.

### Objectives:
- Analyze trends in battery resistance parameters (`Re`, `Rct`, `Battery_Impedance`) over cycles.
- Investigate the effect of temperature on battery performance.
- Visualize key findings and derive insights on battery degradation.

# Data Loading and Inspection

We load the dataset and inspect its structure to identify missing values, column types, and the overall quality of the data

In [27]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

In [18]:
# Set Kaggle environment for dataset download
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()

# Download the dataset
!kaggle datasets download -d patrickfleith/nasa-battery-dataset

# Unzip the dataset
!unzip nasa-battery-dataset.zip -d nasa_battery

Streaming output truncated to the last 5000 lines.
  inflating: nasa_battery/cleaned_dataset/data/02576.csv  
  inflating: nasa_battery/cleaned_dataset/data/02577.csv  
  inflating: nasa_battery/cleaned_dataset/data/02578.csv  
  inflating: nasa_battery/cleaned_dataset/data/02579.csv  
  inflating: nasa_battery/cleaned_dataset/data/02580.csv  
  inflating: nasa_battery/cleaned_dataset/data/02581.csv  
  inflating: nasa_battery/cleaned_dataset/data/02582.csv  
  inflating: nasa_battery/cleaned_dataset/data/02583.csv  
  inflating: nasa_battery/cleaned_dataset/data/02584.csv  
  inflating: nasa_battery/cleaned_dataset/data/02585.csv  
  inflating: nasa_battery/cleaned_dataset/data/02586.csv  
  inflating: nasa_battery/cleaned_dataset/data/02587.csv  
  inflating: nasa_battery/cleaned_dataset/data/02588.csv  
  inflating: nasa_battery/cleaned_dataset/data/02589.csv  
  inflating: nasa_battery/cleaned_dataset/data/02590.csv  
  inflating: nasa_battery/cleaned_dataset/data/02591.csv  
  inf

In [36]:
# Load the dataset
file_path = '/content/nasa_battery/cleaned_dataset/metadata.csv'
data = pd.read_csv(file_path)

# Data Inspection
print("Dataset Information:")
print(data.info())
print("\nFirst 5 Rows:")
print(data.head())


Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7565 entries, 0 to 7564
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   type                 7565 non-null   object
 1   start_time           7565 non-null   object
 2   ambient_temperature  7565 non-null   int64 
 3   battery_id           7565 non-null   object
 4   test_id              7565 non-null   int64 
 5   uid                  7565 non-null   int64 
 6   filename             7565 non-null   object
 7   Capacity             2794 non-null   object
 8   Re                   1956 non-null   object
 9   Rct                  1956 non-null   object
dtypes: int64(3), object(7)
memory usage: 591.1+ KB
None

First 5 Rows:
        type                                         start_time  \
0  discharge  [2010.       7.      21.      15.       0.    ...   
1  impedance  [2010.       7.      21.      16.      53.    ...   
2     charg

# Data Cleaning and Preprocessing
We clean the dataset by:
1. Converting non-numeric columns (`Re`, `Rct`) to numeric values.
2. Calculating `Battery_Impedance` as the sum of `Re` and `Rct`.
3. Parsing `start_time` into a standard datetime format.
4. Adding a `cycle_number` column to track charge-discharge cycles for each battery.

In [41]:
# Convert 'Re' and 'Rct' to numeric (coerce invalid entries to NaN)
data['Re'] = pd.to_numeric(data['Re'], errors='coerce')
data['Rct'] = pd.to_numeric(data['Rct'], errors='coerce')

# Calculate Battery Impedance as the sum of Re and Rct
data['Battery_Impedance'] = data['Re'] + data['Rct']

# Handle missing values
data_cleaned = data.dropna(subset=['Re', 'Rct', 'start_time', 'Battery_Impedance'])

# Parse start_time with a robust function
def parse_start_time(start_time):
    """
    Convert unconventional start_time format into a standard datetime object.
    """
    if isinstance(start_time, str) and "[" in start_time and "]" in start_time:
        # Remove brackets and split into components
        start_time = start_time.replace("[", "").replace("]", "")
        components = start_time.split()
        try:
            # Convert components to numeric
            components = [float(comp) for comp in components]
            # Create datetime object
            if len(components) == 6:
                year, month, day, hour, minute, second = components
                return pd.Timestamp(
                    year=int(year),
                    month=int(month),
                    day=int(day),
                    hour=int(hour),
                    minute=int(minute),
                    second=int(second)
                )
        except Exception as e:
            print(f"Error parsing start_time: {e}")
            return np.nan
    return pd.to_datetime(start_time, errors="coerce")

# Apply the parsing function
data_cleaned["start_time"] = data_cleaned["start_time"].apply(parse_start_time)

# Extract Cycle Number
data_cleaned['cycle_number'] = data_cleaned.groupby('battery_id').cumcount() + 1

# Display the cleaned data
print("Cleaned Dataset:")
print(data_cleaned.info())


Cleaned Dataset:
<class 'pandas.core.frame.DataFrame'>
Index: 1947 entries, 1 to 7560
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   type                 1947 non-null   object        
 1   start_time           1947 non-null   datetime64[ns]
 2   ambient_temperature  1947 non-null   int64         
 3   battery_id           1947 non-null   object        
 4   test_id              1947 non-null   int64         
 5   uid                  1947 non-null   int64         
 6   filename             1947 non-null   object        
 7   Capacity             0 non-null      object        
 8   Re                   1947 non-null   float64       
 9   Rct                  1947 non-null   float64       
 10  Battery_Impedance    1947 non-null   float64       
 11  cycle_number         1947 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(4), object(4)
memory usage: 197.7+ KB


<ipython-input-41-03bff8b77ac1>:40: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-41-03bff8b77ac1>:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Exploratory Data Analysis (EDA)

### Battery Metrics Over Cycles
Here, we plot `Battery_Impedance`, `Re`, and `Rct` against charge-discharge cycles to observe trends.  
Increasing resistance over cycles indicates battery degradation.

In [44]:
# Visualize Battery Impedance, Re, and Rct across cycles
fig = make_subplots(rows=1, cols=3, subplot_titles=("Battery Impedance", "Re", "Rct"))

# Battery Impedance
fig.add_trace(
    go.Scatter(x=data_cleaned['cycle_number'], y=data_cleaned['Battery_Impedance'], mode='lines', name='Battery Impedance'),
    row=1, col=1
)

# Re
fig.add_trace(
    go.Scatter(x=data_cleaned['cycle_number'], y=data_cleaned['Re'], mode='lines', name='Re'),
    row=1, col=2
)

# Rct
fig.add_trace(
    go.Scatter(x=data_cleaned['cycle_number'], y=data_cleaned['Rct'], mode='lines', name='Rct'),
    row=1, col=3
)

fig.update_layout(
    title="Battery Aging: Impedance, Re, and Rct Over Cycles",
    height=600,
    width=1200,
    showlegend=True
)
fig.show()

### Temperature Effect on Battery Parameters

We analyze how ambient temperature influences `Battery_Impedance`, `Re`, and `Rct`.  
Higher temperatures may accelerate the battery's aging process.

In [45]:
# Group data by ambient_temperature and calculate mean values
temperature_effect = data_cleaned.groupby('ambient_temperature').agg(
    avg_battery_impedance=('Battery_Impedance', 'mean'),
    avg_Re=('Re', 'mean'),
    avg_Rct=('Rct', 'mean')
).reset_index()

# Temperature vs Battery Impedance
fig_temp = px.line(
    temperature_effect,
    x='ambient_temperature',
    y=['avg_battery_impedance', 'avg_Re', 'avg_Rct'],
    title="Effect of Temperature on Battery Parameters"
)
fig_temp.update_layout(
    xaxis_title="Ambient Temperature (\u00b0C)",
    yaxis_title="Resistance / Impedance (Ohms)",
    template="plotly_dark"
)
fig_temp.show()

# Correlation Analysis
We examine the correlations between `Re`, `Rct`, and `Battery_Impedance`.  
A strong correlation between these parameters indicates a consistent trend of battery aging.

In [46]:
# Correlation heatmap for Re, Rct, and Battery_Impedance
correlation_matrix = data_cleaned[['Re', 'Rct', 'Battery_Impedance']].corr()
print("Correlation Matrix:")
print(correlation_matrix)

fig_corr = px.imshow(
    correlation_matrix,
    text_auto=True,
    title="Correlation Heatmap for Battery Parameters",
    color_continuous_scale="Viridis"
)
fig_corr.show()

Correlation Matrix:
                    Re  Rct  Battery_Impedance
Re                 1.0 -1.0               -1.0
Rct               -1.0  1.0                1.0
Battery_Impedance -1.0  1.0                1.0


# Observations and Conclusions

## Observations and Conclusions

1. **Battery Aging Trends**:
   - `Battery_Impedance`, `Re`, and `Rct` increase consistently across cycles, indicating battery degradation over time.

2. **Temperature Effects**:
   - Higher ambient temperatures exacerbate battery aging, as seen from the increased resistance values.

3. **Correlation Insights**:
   - `Battery_Impedance` shows strong positive correlations with both `Re` and `Rct`.

### Future Scope:
- Monitor resistance parameters to predict battery life and aging.
- Further analysis with machine learning could help forecast end-of-life for batteries.

# Save Processed Data
We save the cleaned and processed dataset for future analysis and reproducibility.

In [47]:
# Save the cleaned and processed data
data_cleaned.to_csv('cleaned_nasa_battery_data.csv', index=False)
print("Cleaned data saved successfully.")

Cleaned data saved successfully.


# Final Notes
This notebook includes:
1. Data cleaning and preprocessing.
2. Visualizations for trends and temperature analysis.
3. Correlation analysis for insights.
4. Conclusions and recommendations.

Thank you for exploring the NASA Battery Dataset!